In [0]:
%run "DatabricksMasterclass/ConnectToDataLake"

Connection to datalakeshiva88 is successful


In [0]:
print(v_secretKey)

[REDACTED]


[FileInfo(path='abfss://source@datalakeshiva88.dfs.core.windows.net/Sales.csv', name='Sales.csv', size=869537, modificationTime=1757579335000)]

df_sales hs the data from Sales.csv


# DELTA LAKE

In [0]:
# Command to run another file in the same directory
# This will run the file and all variable and objects will be made available in the same folder

# %run "DatabricksMasterclass/Tutorial"

In [0]:
df_sales.write.format("delta")\
    .mode('append')\
        .option('path','abfss://destination@datalakeshiva88.dfs.core.windows.net/sales')\
            .save()

## Managed vs External Delta Tables

In [0]:
%sql
--Create Database

create database if not exists salesDB;


--spark.sql("CREATE DATABASE IF NOT EXISTS shiva")

**Managed Table**

In [0]:
%sql

create table if not exists salesDB.mantable
(
  id int,
  name string,
  marks int
)
using DELTA



In [0]:
%sql
insert into salesDB.mantable
values
(1,'aa',30),
(2,'aa',33),
(3,'aa',35),
(4,'aa',40);


num_affected_rows,num_inserted_rows
4,4


In [0]:
%sql
select * from salesdb.mantable;

id,name,marks
1,aa,30
2,aa,33
3,aa,35
4,aa,40


In [0]:
%sql
--drop table salesdb.mantable;

**External Table**


Following Step1 and Step2 have failed.

Resolution is - check https://www.youtube.com/watch?v=kRfNXFh9T3U
- create a new Access Connector for Azure Databricks "ac-extstorage-eastus" in the same eastus region where databricks is available
- provide/grant "Storage Blob Data Contributor" access to "ac-extstorage-eastus" on ADLS Gen2 Storage Account "datalakeshiva88" IAM. And also give access to unity-catalog access connector
- Open Databricks > catalog - External Data > Credentials  > extstorage-credential
- Open Databricks > catalog - External Data > External Locations  > extstorage-location

Then directly execute the Step3

In [0]:
%sql

-- Step 1: Create a storage credential (if you do not have one)
--You must run CREATE STORAGE CREDENTIAL in a Databricks SQL warehouse, not on a cluster. This command is not supported on interactive or all-purpose clusters, which is why you see a syntax error.
--Fix:
--Switch your compute to a Databricks SQL warehouse in the Databricks SQL editor, then run your code there.


/*
CREATE STORAGE CREDENTIAL shiva_datalake_cred
WITH AZURE_SERVICE_PRINCIPAL (
   CLIENT_ID = 'Application (client) ID',
   CLIENT_SECRET = v_secretKey,
   TENANT_ID = 'Directory (tenant) ID'
 )
STORAGE_URL = 'https://datalakeshiva88.dfs.core.windows.net/'
;
*/

In [0]:
%sql
-- Step 2: Register the parent path as an external location

/*
CREATE EXTERNAL LOCATION IF NOT EXISTS my_ext_location
URL 'abfss://destination@datalakeshiva88.dfs.core.windows.net/salesDB'
WITH (STORAGE CREDENTIAL shiva_datalake_cred);
*/



In [0]:
%sql
-- Step 3: Create the external table 
CREATE TABLE salesDB.exttable
(
  id int,
  name string,
  marks int
)
USING DELTA
LOCATION 'abfss://destination@datalakeshiva88.dfs.core.windows.net/salesDB/exttable'


In [0]:
%sql
insert into salesDB.exttable
values
(1,'aa',30),
(2,'aa',33),
(3,'aa',35),
(4,'aa',40);


num_affected_rows,num_inserted_rows
4,4


In [0]:
%sql
select * from salesdb.exttable;

id,name,marks
1,aa,30
2,aa,33
3,aa,35
4,aa,40


In [0]:
%sql
--drop table salesdb.exttable;

-- This has dropped the table from Databricks only and not from the datalakeshiva88

## Delta Tables Functionalities

**Insert**

In [0]:
%sql
insert into salesDB.exttable
values
(5,'aa',30),
(6,'aa',33),
(7,'aa',35),
(8,'aa',40);


num_affected_rows,num_inserted_rows
4,4


**Delete Data**

In [0]:
%sql

delete from salesDB.exttable where id = 8;

num_affected_rows
1


**Data Versioning**

In [0]:
%sql

describe history salesDB.exttable;

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
4,2025-09-15T17:21:31Z,143896217133723,biradar.shiv88@gmail.com,OPTIMIZE,"Map(predicate -> [], auto -> true, clusterBy -> [], zOrderBy -> [], batchId -> 0)",null,List(2152550817360483),0912-092435-ifhk5xlm,3,SnapshotIsolation,false,"Map(numRemovedFiles -> 2, numRemovedBytes -> 2004, p25FileSize -> 1065, numDeletionVectorsRemoved -> 1, minFileSize -> 1065, numAddedFiles -> 1, maxFileSize -> 1065, p75FileSize -> 1065, p50FileSize -> 1065, numAddedBytes -> 1065)",null,Databricks-Runtime/16.4.x-photon-scala2.12
3,2025-09-15T17:21:25Z,143896217133723,biradar.shiv88@gmail.com,DELETE,"Map(predicate -> [""(id#19 = 8)""])",null,List(2152550817360483),0912-092435-ifhk5xlm,2,WriteSerializable,false,"Map(numRemovedFiles -> 0, numRemovedBytes -> 0, numCopiedRows -> 0, numDeletionVectorsAdded -> 1, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 13085, numDeletionVectorsUpdated -> 0, numDeletedRows -> 1, scanTimeMs -> 10985, numAddedFiles -> 0, numAddedBytes -> 0, rewriteTimeMs -> 2075)",null,Databricks-Runtime/16.4.x-photon-scala2.12
2,2025-09-15T07:58:32Z,143896217133723,biradar.shiv88@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(2152550817360483),0912-092435-ifhk5xlm,1,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 4, numOutputBytes -> 1002)",null,Databricks-Runtime/16.4.x-photon-scala2.12
1,2025-09-15T07:56:12Z,143896217133723,biradar.shiv88@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(2152550817360483),0912-092435-ifhk5xlm,0,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 4, numOutputBytes -> 1002)",null,Databricks-Runtime/16.4.x-photon-scala2.12
0,2025-09-15T07:55:46Z,143896217133723,biradar.shiv88@gmail.com,CREATE TABLE,"Map(partitionBy -> [], clusterBy -> [], description -> null, isManaged -> false, properties -> {""delta.enableDeletionVectors"":""true""}, statsOnLoad -> false)",null,List(2152550817360483),0912-092435-ifhk5xlm,null,WriteSerializable,true,Map(),null,Databricks-Runtime/16.4.x-photon-scala2.12


**TIME TRAVEL  or Rollback to a specific time back**

In [0]:
%sql
restore table salesDB.exttable to version as of 2;
    
select * from salesDB.exttable;

id,name,marks
1,aa,30
2,aa,33
3,aa,35
4,aa,40
5,aa,30
6,aa,33
7,aa,35
8,aa,40


In [0]:
%sql
select * from salesDB.exttable;

id,name,marks
1,aa,30
2,aa,33
3,aa,35
4,aa,40
5,aa,30
6,aa,33
7,aa,35
8,aa,40


**VACUUM** 
This will all the history of parquet files and retains only the latest one with latest data and no version history will be maintained

In [0]:
%sql

-- vacuum salesDB.exttable; -- This will delete only data/parquet files older than 7 days
-- vacuum salesDB.exttable retain 0 hours; -- This will delete all the data/parquet files till now and retains only the latest one parquet file only
    
select * from salesDB.exttable;

### DELTA Table Optimization

**OPTIMIZE**

In [0]:
%sql
optimize salesDB.exttable;
-- This enhances the performance by 30 to 40% for larger tables, since this is a smaller table hence 1.99 sec to 1.91 seconds only 

path,metrics
abfss://destination@datalakeshiva88.dfs.core.windows.net/salesDB/exttable,"List(1, 2, List(1070, 1070, 1070.0, 1, 1070), List(1002, 1002, 1002.0, 2, 2004), 0, null, null, 0, 1, 2, 0, true, 0, 0, 1757959053468, 1757959056801, 4, 1, null, List(0, 0), null, 3, 3, 210, 0, null)"


**ZORDER BY**

In [0]:
%sql
optimize salesDB.exttable zorder by (id);

-- optimize reduces the number of blocks/parquet files from 4 to 2
-- zorder by -- orders the complete data in the 2 blocks based on id column, if id=3 is found in the first block then it will not read the data from 2nd block, hence increasing the performance by reducing the number of blocks being read.

path,metrics
abfss://destination@datalakeshiva88.dfs.core.windows.net/salesDB/exttable,"List(0, 0, List(null, null, 0.0, 0, 0), List(null, null, 0.0, 0, 0), 0, List(minCubeSize(107374182400), List(0, 0), List(1, 1070), 0, List(0, 0), 0, null), null, 0, 0, 1, 1, false, 0, 0, 1757960051818, 1757960052308, 4, 0, null, List(0, 0), null, 3, 3, 0, 0, null)"


In [0]:
%sql
select * from salesDB.exttable ;

id,name,marks
1,aa,30
2,aa,33
3,aa,35
4,aa,40
5,aa,30
6,aa,33
7,aa,35
8,aa,40


# Incremental Loading with AUTO LOADER
## Uses Streaming Dataframe to populate the parquet files from source to destination incrementally

In [0]:
df = spark.readStream.format("cloudFiles")\
                      .option("cloudFiles.format", "parquet")\
                      .option("cloudFiles.schemaLocation", "abfss://autoloader-destination@datalakeshiva88.dfs.core.windows.net/checkpoint")\
                      .load("abfss://autoloader-source@datalakeshiva88.dfs.core.windows.net")

In [0]:
df.writeStream.format('delta')\
    .option('checkpointLocation', 'abfss://autoloader-destination@datalakeshiva88.dfs.core.windows.net/checkpoint')\
    .trigger(processingTime='5 seconds')\
    .start('abfss://autoloader-destination@datalakeshiva88.dfs.core.windows.net/data')

# Click on interrupt or stop button after some time, otherwise this will keep running in compute instance incuring heavy charges for running the compute instance 